In [1]:
!pip install -q datasets transformers qwen_vl_utils accelerate


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [14]:
import io
import json
from PIL import Image
from datasets import load_dataset
import torch
from transformers import AutoModelForVision2Seq, AutoProcessor, Qwen2VLProcessor
#from trl import SFTConfig, SFTTrainer
#from peft import LoraConfig
from qwen_vl_utils import process_vision_info



In [67]:
system_message = """당신은 주어진 원재료 이름 리스트로 부터 알러지 유발 성분이 있는지 분석하는 전문적인 의사입니다.

다음의 지시사항을 따르십시오.
1. 주어진 원재료명에 대한 것만 대답하세요
2. 원재료명과 관련없는 것은 아무것도 하지마세요
3. 원재료명에 대한 정보가 없는 경우 "알 수 없는 재료명입니다"라고 답해주세요
4. 원재료명에 대해서 알러지 다음 19가지 항목에 대해서 알러지 유발 가능성이 있는지 판별하여 해당하는 모든 카테고리에 원재료명을 출력하세요
5. 19가지항목에 해당하지 않으면 맨마지막에 "그외" 항목에 빠짐없이 출력해주세요

# 19가지항목:
-----------
난류(계란류,가금류)
소고기
돼지고기
닭고기
새우
게
오징어
고등어
조개류(굴,전복,홍합포함)
우유
땅콩
호두
잣
대두
복숭아
토마토
밀
메밀
아황산류
"""

# 사용자 입력과 예시 형태를 합친 하나의 프롬프트
prompt_template = """# 원재료명:
-----
{itemlist}

알러지유발 가능성이 있는 원재료명에 대해서 **아래 예시 형태**를 반드시 지켜서 출력해주세요.
# 예시 :
-------
난류(계란류,가금류) : [[원재료명 리스트]]
소고기 : [[원재료명 리스트]]
돼지고기 : [[원재료명 리스트]]
닭고기 : [[원재료명 리스트]] 
새우 : [[원재료명 리스트]]
게 : [[원재료명 리스트]]
오징어 : [[원재료명 리스트]]
고등어 : [[원재료명 리스트]]
조개류(굴,전복,홍합포함) : [[원재료명 리스트]]
우유 : [[원재료명 리스트]]
땅콩 : [[원재료명 리스트]]
호두 : [[원재료명 리스트]]
잣 : [[원재료명 리스트]]
대두 : [[원재료명 리스트]]
복숭아 : [[원재료명 리스트]]
토마토 : [[원재료명 리스트]]
밀 : [[원재료명 리스트]]
메밀 : [[원재료명 리스트]]
아황산류 : [[원재료명 리스트]]
그외 : [[원재료명 리스트]]
"""

In [68]:

def format_data(itemlist, txt_path):
   return {
       "messages": [
           {
               "role": "system",
               "content": [
                   {
                       "type": "text",
                       "text": system_message,
                   }
               ],
           },
           {
               "role": "user",
               "content": [
                   {
                       "type": "text",
                       "text": prompt_template.format(itemlist=itemlist),
                   },  
               ],
           },
           {
               "role": "assistant",
               "content": [
                   {
                       "type": "text",
                       "text": "",
                   }
               ],
           },
       ],
      "rst_file_name": txt_path,
   }

In [69]:
import os

folder_path = "./data"
train_list= []

for file_name in os.listdir(folder_path):
  
  if file_name.lower().endswith("_new.txt"):
    #print("file_name=",file_name)
    # 파일의 전체 경로를 만듭니다.
    full_path = os.path.join(folder_path, file_name)

    with open(full_path, 'r', encoding='utf-8') as f:
        # 1. 파일의 모든 내용을 한 번에 읽기
        content = f.read()
        #print("content=",content)
        f.close()
    
    # 1. 파일명에서 확장자를 제외한 부분(stem)을 가져옵니다.
    file_stem, _ = os.path.splitext(file_name)
    
    # 2. 새로운 파일명(original_name_ocr.txt)을 만듭니다.
    new_file_name = f"{file_stem}_result.txt"
    
    # 3. 원래 파일의 폴더 경로와 새로운 파일명을 합쳐서 새 경로를 만듭니다.
    txt_path = os.path.join(folder_path, new_file_name)
    
    train_list.append(format_data(content, txt_path))
    


In [70]:
print(len(train_list), train_list[2])

62 {'messages': [{'role': 'system', 'content': [{'type': 'text', 'text': '당신은 주어진 원재료 이름 리스트로 부터 알러지 유발 성분이 있는지 분석하는 전문적인 의사입니다.\n\n다음의 지시사항을 따르십시오.\n1. 주어진 원재료명에 대한 것만 대답하세요\n2. 원재료명과 관련없는 것은 아무것도 하지마세요\n3. 원재료명에 대한 정보가 없는 경우 "알 수 없는 재료명입니다"라고 답해주세요\n4. 원재료명에 대해서 알러지 다음 19가지 항목에 대해서 알러지 유발 가능성이 있는지 판별하여 해당하는 모든 카테고리에 원재료명을 출력하세요\n5. 19가지항목에 해당하지 않으면 맨마지막에 "그외" 항목에 빠짐없이 출력해주세요\n\n# 19가지항목:\n-----------\n난류(계란류,가금류)\n소고기\n돼지고기\n닭고기\n새우\n게\n오징어\n고등어\n조개류(굴,전복,홍합포함)\n우유\n땅콩\n호두\n잣\n대두\n복숭아\n토마토\n밀\n메밀\n아황산류\n'}]}, {'role': 'user', 'content': [{'type': 'text', 'text': '# 원재료명:\n-----\n\n#결과\n원재료명:\n마요네즈(대두유, 달걀, 밀, 우유, 대두, 땅콩, 복숭아, 토마토, 고등어, 게, 새우, 돼지고기, 호두, 오징어, 조개류, 아황산류, 잣, 제품과 같은 제조시설에서 제조하고 있습니다.)\n\n혼입가능, 같은제조시설 :\n복숭아\n밀\n우유\n대두\n땅콩\n달걀\n토마토\n\n알러지유발 가능성이 있는 원재료명에 대해서 **아래 예시 형태**를 반드시 지켜서 출력해주세요.\n# 예시 :\n-------\n난류(계란류,가금류) : [[원재료명 리스트]]\n소고기 : [[원재료명 리스트]]\n돼지고기 : [[원재료명 리스트]]\n닭고기 : [[원재료명 리스트]] \n새우 : [[원재료명 리스트]]\n게 : [[원재료명 리스트]]\n오징어 : [[원재료명 리스트]]\n고등어 : [[원재료명 리스트]]\n조개류

In [19]:
model_id = "Qwen/Qwen2.5-VL-7B-Instruct"

model = AutoModelForVision2Seq.from_pretrained(
   model_id,
   device_map="auto",                            # GPU 메모리에 자동 할당
   torch_dtype=torch.bfloat16,                   # bfloat16 정밀도 사용
)

processor = AutoProcessor.from_pretrained(model_id)  # 텍스트/이미지 전처리기 로드

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/modeling_auto.py:2242: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [71]:
sample_data = train_list[0]['messages']

sample_prompt = processor.apply_chat_template(
    sample_data,
    tokenize=False,
    add_generation_prompt=False,
)

In [72]:
def split_input_and_label(prompt):
    input = prompt.split('<|im_start|>assistant')[0] + '<|im_start|>assistant'
    label = prompt.split('<|im_start|>assistant')[1]
    return input, label

In [73]:
sample_prompt_input, sample_prompt_label = split_input_and_label(sample_prompt)

In [74]:
sample_data_image_inputs, sample_data_video_inputs = process_vision_info(sample_data)

In [75]:
inputs = processor(
    text=[sample_prompt_input],
    images=sample_data_image_inputs,
    videos=sample_data_video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

In [76]:
%%time
# Inference: Generation of the output
generated_ids = model.generate(
    **inputs,
    max_new_tokens=2048,
    temperature=0.1,
    top_p=0.001,
    repetition_penalty=1.05,
)

generated_ids_trimmed = [out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)]
output_text = processor.batch_decode(generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False)
print("추론결과=\n")
#print(output_text)


추론결과=

CPU times: user 52 s, sys: 1.49 s, total: 53.5 s
Wall time: 53.5 s


In [77]:
print(output_text[0])


------
밀 : [[밀가루(밀:미국산)]]
대두 : [[대두]]
우유 : [[우지(호주산), 아자유(외국산):필리핀, 인도네시아, 말레이시아 등]]
그외 : [[팜유(말레이시아산), 팜스테아린유, 레시틴, 가공유지(팜핵에스테르화유:말레이시아산), 쇼트닝, 쇼트닝II, 맥아분말, 천일염, 안데스 소금(아르헨티나산), 체다치즈분말, 향료, 효소제2종]]


In [78]:
sample_data

[{'role': 'system',
  'content': [{'type': 'text',
    'text': '당신은 주어진 원재료 이름 리스트로 부터 알러지 유발 성분이 있는지 분석하는 전문적인 의사입니다.\n\n다음의 지시사항을 따르십시오.\n1. 주어진 원재료명에 대한 것만 대답하세요\n2. 원재료명과 관련없는 것은 아무것도 하지마세요\n3. 원재료명에 대한 정보가 없는 경우 "알 수 없는 재료명입니다"라고 답해주세요\n4. 원재료명에 대해서 알러지 다음 19가지 항목에 대해서 알러지 유발 가능성이 있는지 판별하여 해당하는 모든 카테고리에 원재료명을 출력하세요\n5. 19가지항목에 해당하지 않으면 맨마지막에 "그외" 항목에 빠짐없이 출력해주세요\n\n# 19가지항목:\n-----------\n난류(계란류,가금류)\n소고기\n돼지고기\n닭고기\n새우\n게\n오징어\n고등어\n조개류(굴,전복,홍합포함)\n우유\n땅콩\n호두\n잣\n대두\n복숭아\n토마토\n밀\n메밀\n아황산류\n'}]},
 {'role': 'user',
  'content': [{'type': 'text',
    'text': '# 원재료명:\n-----\n\n#결과\n원재료명:\n밀가루(밀:미국산), 흔합식용유[팜유(말레이시아산), 아자유(외국산):필리핀, 인도네시아, 말레이시아 등), 토코페롤(혼합형)], 쇼트닝[가공유지(팜핵에스테르화유:말레이시아산), 우지(호주산), 팜유, 팜스테아린유, 레시틴], 설탕, 기타과당, 산도조절제3종, 쇼트닝II, 맥아분말, 천일염, 안데스 소금(아르헨티나산), 체다치즈분말, 레시틴, 향료, 효소제2종\n\n혼입가능, 같은제조시설:\n밀, 대두, 쇠고기, 우유 함유\n\n알러지유발 가능성이 있는 원재료명에 대해서 **아래 예시 형태**를 반드시 지켜서 출력해주세요.\n# 예시 :\n-------\n난류(계란류,가금류) : [[원재료명 리스트]]\n소고기 : [[원재료명 리스트]]\n돼지고기 : [[원재료명 리스트]]\n닭고기 : [[원재료명 리스트]]

In [79]:
%%time
from tqdm import tqdm

for index, item in enumerate(tqdm(train_list)):
    test_data = item['messages']
    txt_path = item['rst_file_name']
    #print("처리중 txt_path=", txt_path)

    test_prompt = processor.apply_chat_template(
        test_data,
        tokenize=False,
        add_generation_prompt=False,
    )
    
    test_prompt_input, test_prompt_label = split_input_and_label(test_prompt)
    test_data_image_inputs, test_data_video_inputs = process_vision_info(test_data)
    
    inputs = processor(
        text=[test_prompt_input],
        images=test_data_image_inputs,
        videos=test_data_video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to("cuda")
    
    # Inference: Generation of the output
    generated_ids = model.generate(
        **inputs,
        max_new_tokens=2048,
        temperature=0.1,
        top_p=0.001,
        repetition_penalty=1.05,
    )

    generated_ids_trimmed = [out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)]
    output_text = processor.batch_decode(generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    #print("추론결과=\n")
    #print(output_text)    
    
    # 새 .txt 파일을 생성하고 내용을 씁니다.
    with open(txt_path, "w", encoding="utf-8") as f:
        f.write(output_text[0])
        f.close()

    #print(f"'{txt_path}' 파일이 생성되었습니다.")   

100%|██████████| 62/62 [1:23:59<00:00, 81.29s/it] 

CPU times: user 1h 21min 48s, sys: 2min 7s, total: 1h 23min 55s
Wall time: 1h 23min 59s
